In [1]:
import csv
import warnings
import scipy as sp
import numpy as np
import pandas as pd
from datetime import datetime
warnings.filterwarnings("ignore")
from pandas_plink import read_plink1_bin

In [2]:
experiments = ['34.1.1']

pops_list = ['EurOnly','EurEas','EurEasAmr','EurEasAmrAfr']

In [5]:
for pops_trio in pops_list:
    
    print(pops_trio)
    
    for exp_num in experiments:
    
        #Read Auxiliary Population Summary Statistics
        ss_df_aux1 = pd.read_csv('/Users/ritwizkamal/BIRDS_server/ForGithub/example_data/aux_one'+str(exp_num)+\
                                 '.sumstat', delimiter=' ', header=None,names=['chr','snp',\
                                'pos','a0','a1','beta','se','t','pvalue','n'])

        ss_df_aux2 = pd.read_csv('/Users/ritwizkamal/BIRDS_server/ForGithub/example_data/aux_two'+str(exp_num)+\
                                 '.sumstat', delimiter=' ', header=None, names=['chr','snp',\
                                'pos','a0','a1','beta','se','t','pvalue','n'])

        ss_df_aux3 = pd.read_csv('/Users/ritwizkamal/BIRDS_server/ForGithub/example_data/aux_three'+str(exp_num)+\
                                 '.sumstat', delimiter=' ', header=None, names=['chr','snp',\
                                'pos','a0','a1','beta','se','t','pvalue','n'])

        ss_df_aux4 = pd.read_csv('/Users/ritwizkamal/BIRDS_server/ForGithub/example_data/aux_four'+str(exp_num)+\
                                 '.sumstat', delimiter=' ', header=None, names=['chr','snp',\
                                'pos','a0','a1','beta','se','t','pvalue','n'])

        #Read Target Population Summary Statistics
        ss_df_tar = pd.read_csv('/Users/ritwizkamal/BIRDS_server/ForGithub/example_data/target'+str(exp_num)+\
                                '.sumstat', delimiter=' ', header=None, names=['chr','snp',\
                                'pos','a0','a1','beta','se','t','pvalue','n'])

        #Read Target Population Genotype Data (Ref./LD File)
        G_sas = read_plink1_bin('/Users/ritwizkamal/BIRDS_server/1000Genomes_MAGMA/g1000_sas/sas_chr22_1maf_geno.bed',\
                                '/Users/ritwizkamal/BIRDS_server/1000Genomes_MAGMA/g1000_sas/sas_chr22_1maf_geno.bim',\
                                '/Users/ritwizkamal/BIRDS_server/1000Genomes_MAGMA/g1000_sas/sas_chr22_1maf_geno.fam',\
                                verbose = False)
        X_sas = G_sas.values
        X_sas = (X_sas - np.mean(X_sas, axis=0))/np.std(X_sas, axis=0)
        all_snps_sas_ld_full = list(ss_df_tar['snp'])

        common_snps = list(set(ss_df_aux1.iloc[:,1]) & set(ss_df_aux2.iloc[:,1]) &\
                           set(ss_df_aux3.iloc[:,1]) & set(ss_df_aux4.iloc[:,1]) &\
                           set(ss_df_tar.iloc[:,1]) & set(all_snps_sas_ld_full))

        #Preprocess Target and Auxiliary Summ. Stats. File
        ss_df_tar = ss_df_tar[ss_df_tar["snp"].isin(common_snps)]
        ss_df_aux1 = ss_df_aux1[ss_df_aux1["snp"].isin(common_snps)]
        ss_df_aux2 = ss_df_aux2[ss_df_aux2["snp"].isin(common_snps)]
        ss_df_aux3 = ss_df_aux3[ss_df_aux3["snp"].isin(common_snps)]
        ss_df_aux4 = ss_df_aux4[ss_df_aux4["snp"].isin(common_snps)]
        
        N_tar = int(ss_df_tar['n'][0])
        N_aux1 = int(ss_df_aux1['n'][0])
        N_aux2 = int(ss_df_aux2['n'][0])
        N_aux3 = int(ss_df_aux3['n'][0])
        N_aux4 = int(ss_df_aux4['n'][0])

        tar_snps = list(ss_df_tar['snp'])
        tar_a0 = list(ss_df_tar['a0'])
        tar_a1 = list(ss_df_tar['a1'])
        tar_pvals = np.asarray(list(ss_df_tar['pvalue']), dtype='float')
        tar_betas = np.asarray(list(ss_df_tar['beta']), dtype='float')
        tar_chroms = list(ss_df_tar['chr'])
        tar_pos = np.asarray(list(ss_df_tar['pos']), dtype='float')
        tar_corr = np.asarray(list(ss_df_tar['beta']), dtype='float')*N_tar
        
        aux1_snps = list(ss_df_aux1['snp'])
        aux1_betas = np.asarray(list(ss_df_aux1['beta']), dtype='float')

        aux2_snps = list(ss_df_aux2['snp'])
        aux2_betas = np.asarray(list(ss_df_aux2['beta']), dtype='float')
        
        aux3_snps = list(ss_df_aux3['snp'])
        aux3_betas = np.asarray(list(ss_df_aux3['beta']), dtype='float')
        
        aux4_snps = list(ss_df_aux4['snp'])
        aux4_betas = np.asarray(list(ss_df_aux4['beta']), dtype='float')

        #Preprocess Target Ref. file
        temp_set = set(tar_snps)
        temp_indices_tar = [i for i, e in enumerate(all_snps_sas_ld_full) if e in temp_set]
        all_snps_sas_ld_full = [i for i in all_snps_sas_ld_full if i in common_snps]
        X_tar = X_sas[:, temp_indices_tar]
        X_tar = (X_tar - np.mean(X_tar, axis=0))/np.std(X_tar, axis=0)

        for i in range(len(aux1_betas)):
            #Choose according to pops in aux pops
            if pops_trio == 'EurOnly':
                temp = (aux1_betas*1.0) + (aux2_betas*0.0) + (aux3_betas*0.0) + (aux4_betas*0.0)
            elif pops_trio == 'EasOnly':
                temp = (aux1_betas*0.0) + (aux2_betas*1.0) + (aux3_betas*0.0) + (aux4_betas*0.0)
            elif pops_trio == 'AmrOnly':
                temp = (aux1_betas*0.0) + (aux2_betas*0.0) + (aux3_betas*1.0) + (aux4_betas*0.0)
            elif pops_trio == 'AfrOnly':
                temp = (aux1_betas*0.0) + (aux2_betas*0.0) + (aux3_betas*0.0) + (aux4_betas*1.0)

            elif pops_trio == 'EurAmr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas*per_pop_weight[0]) + (aux2_betas*0.0) +\
                (aux3_betas*per_pop_weight[1]) + (aux4_betas*0.0)
            elif pops_trio == 'EurEas':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas*per_pop_weight[0]) + (aux2_betas*per_pop_weight[1]) +\
                (aux3_betas*0.0) + (aux4_betas*0.0)
            elif pops_trio == 'EurAfr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas*per_pop_weight[0]) + (aux2_betas*0.0) +\
                (aux3_betas*0.0) + (aux4_betas*per_pop_weight[1])
            elif pops_trio == 'EasAmr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas*0.0) + (aux2_betas*per_pop_weight[0]) +\
                (aux3_betas*per_pop_weight[1]) + (aux4_betas*0.0)
            elif pops_trio == 'EasAfr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas*0.0) + (aux2_betas*per_pop_weight[0]) +\
                (aux3_betas*0.0) + (aux4_betas*per_pop_weight[1])
            elif pops_trio == 'AmrAfr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas*0.0) + (aux2_betas*0.0) +\
                (aux3_betas*per_pop_weight[0]) + (aux4_betas*per_pop_weight[1])

            elif pops_trio == 'EurEasAmr':
                per_pop_weight = [0.33,0.33,0.33]
                temp = (aux1_betas*per_pop_weight[0]) + (aux2_betas*per_pop_weight[1]) +\
                (aux3_betas*per_pop_weight[2]) + (aux4_betas*0.0)
            elif pops_trio == 'EurEasAfr':
                per_pop_weight = [0.33,0.33,0.33]
                temp = (aux1_betas*per_pop_weight[0]) + (aux2_betas*per_pop_weight[1]) +\
                (aux3_betas*0.0) + (aux4_betas*per_pop_weight[2])
            elif pops_trio == 'EurAmrAfr':
                per_pop_weight = [0.33,0.33,0.33]
                temp = (aux1_betas*per_pop_weight[0]) + (aux2_betas*0.0) +\
                (aux3_betas*per_pop_weight[1]) + (aux4_betas*per_pop_weight[2])
            elif pops_trio == 'EasAmrAfr':
                per_pop_weight = [0.33,0.33,0.33]
                temp = (aux1_betas*0.0) + (aux2_betas*per_pop_weight[0]) +\
                (aux3_betas*per_pop_weight[1]) + (aux4_betas*per_pop_weight[2])

            elif pops_trio == 'EurEasAmrAfr':
                per_pop_weight = [0.25,0.25,0.25,0.25]
                temp = (aux1_betas*per_pop_weight[0]) + (aux2_betas*per_pop_weight[1]) + \
                (aux3_betas*per_pop_weight[2]) + (aux4_betas*per_pop_weight[3])
            else:
                print("Invalid Aux Pops!")

            aux_betas = temp
            
        P = len(common_snps)

        #Initialize Beta_hats
        Beta_hat_init = np.random.normal(0, 0.00000000001, len(tar_snps))
        
        L1_penalty = 1000
        L2_penalty = 0.00000005

        XtX = np.matmul(X_tar.T, X_tar)
        XtX = abs(1-L2_penalty)*XtX
        r = tar_corr
        
        mu = 0.1

        def func(Bs, XtX, r, L1_penalty, L2_penalty):
            term1 = np.matmul(np.matmul(Bs.T, XtX), Bs)
            term2 = 2*np.matmul(Bs.T, r)
            term3 = L2_penalty*np.matmul(Bs.T,Bs)
            temp = Bs - aux_betas
            nesterov = mu*np.log((0.5*np.exp(-temp/mu))+(0.5*np.exp(temp/mu)))
            term4 = L1_penalty*sum(nesterov)
            return term1-term2+term3+term4

        def jac_func(Bs, XtX, r, L1_penalty, L2_penalty):
            term1 = 2*np.matmul(XtX,Bs)
            term2 = 2*r
            term3 = L2_penalty*2*Bs
            temp = Bs - aux_betas
            nesterov = np.divide((-np.exp(-temp/mu) + np.exp(temp/mu)),(np.exp(-temp/mu) + np.exp(temp/mu)))
            term4 = L1_penalty*nesterov
            return term1-term2+term3+term4

        print(datetime.now())
        ans = sp.optimize.minimize(func, jac=jac_func, x0=Beta_hat_init,\
                                   args=(XtX, r, L1_penalty, L2_penalty),method='L-BFGS-B')
        print(datetime.now())
        
        final_snps = tar_snps
        final_betas = ans.x
        final_chr = tar_chroms
        final_pos = tar_pos
        final_a0 = tar_a0
        final_a1 = tar_a1
        final_results_df = pd.DataFrame({'CHR':final_chr, 'SNP':final_snps, 'POS':final_pos,\
                                         'A0':final_a0, 'A1':final_a1, 'BETA':final_betas,})

        final_results_df.to_csv('/Users/ritwizkamal/BIRDS_server/ForGithub/example_data/Results/mppV2_pred_tar_betas'+\
                                '_'+str(pops_trio)+str(exp_num)+'.txt', sep='\t', index=False)

        print(exp_num+" ... Done!")

EurOnly
2024-12-01 23:05:52.821563
2024-12-01 23:05:59.061783
34.1.1 ... Done!
EurEas
2024-12-01 23:06:00.722787
2024-12-01 23:06:07.088135
34.1.1 ... Done!
EurEasAmr
2024-12-01 23:06:08.624910
2024-12-01 23:06:14.918526
34.1.1 ... Done!
EurEasAmrAfr
2024-12-01 23:06:17.125365
2024-12-01 23:06:23.370318
34.1.1 ... Done!
